In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
plt.style.use("seaborn-whitegrid")

C:\Users\Benny Boi\AppData\Local\Temp\ipykernel_3224\3963306264.py:1: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn-whitegrid")


In [3]:
pd.options.display.max_columns = 500
pd.options.display.max_rows = 4000

# Retail Dataset

### Data holds the basic information about sales data. The company have sales agencies / resellers and branches and the data file holds only the branch/reseller information in the customer field.

    -InvoiceID : ID of the transaction. A transaction might hold multiple records for the same customer at the same date with multiple products (SKU). DocumentID might be useful for combining the transactions and detecting the items sold together.
    -Date : Date of transaction / sell. In the date time format.
    -ProductID : Item / Product code. The unique code for each item sold.
    -TotalSales : Sales price for the transaction. If you want to get unit_price , divide TotalSales column to Quantity column
    -Discount : Discount amount for the transaction.
    -CustomerID : Unique customer id for each customer. For the data set, customer can be a reseller or a branch of the company.
    -Quantity : Number of items sold in the transaction.

<br><b>How to Use Data Set ?</br></b>
Some of the potential questions from the data might be listed as below, please add more to discussion if you have any idea:

<br><b>What is the optimum price for each item?</br></b>

    -Demand prediction for each item on next months.
    -Customer Lifetime Value for each Customer.
    -Customer Segmentation (an easy approach of RFM or more complex segmentations)
    -Customer <> Product Recommendation (what are the best products for the customers).

In [5]:
retail = pd.read_csv(r"C:\Users\Benny Boi\Repositories\Medoci\data\kaggle\competitions\clv_retail\retail.csv")

In [7]:
retail.shape

(29103, 8)

In [8]:
retail.columns

Index(['Unnamed: 0', 'InvoiceID', 'Date', 'ProductID', 'TotalSales',
       'Discount', 'CustomerID', 'Quantity'],
      dtype='object')

In [6]:
retail.head()

,Unnamed: 0,InvoiceID,Date,ProductID,TotalSales,Discount,CustomerID,Quantity
0,0,328,2019-12-27,1684,796.610169,143.389831,185,4
1,1,329,2019-12-27,524,355.932203,64.067797,185,2
2,2,330,2019-12-27,192,901.694915,162.305085,230,4
3,3,330,2019-12-27,218,182.754237,32.895763,230,1
4,4,330,2019-12-27,247,780.101695,140.418305,230,4
